In [1]:
%matplotlib inline
import pandas as pd

In [7]:
data = pd.read_csv('./data/data_csv.csv')
data.head(2000)

,Country Name,Country Code,Year,Value
0,Albania,ALB,2011,21.74
1,Algeria,DZA,2012,22.08
2,Argentina,ARG,2005,16.74
3,Argentina,ARG,2010,23.75
4,Argentina,ARG,2013,23.68
5,Armenia,ARM,2004,24.03
6,Armenia,ARM,2008,21.67
7,Australia,AUS,2006,20.83
8,Austria,AUT,2009,16.94
9,Azerbaijan,AZE,2008,25.42


In [8]:
#Plotting the trend of unpaid labor in a few countries per continent 
countries_NA = ['CAN','USA','MEX']
countries_SA = ['BRA','CRI','ARG']
countries_AUS = ['AUS','NZL']
countries_ASIA = ['IND','MNG','CHN']
countries_EUR = ['GBR','FRA','GER']
countries_AFR = ['DZA','RSA','TUN']